In [13]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import tqdm
import asyncio
import json

In [97]:
old_posts = pd.read_parquet("app_files/lw_data.parquet")

In [98]:
old_posts.columns

Index(['_id', 'Unnamed: 0', 'title', 'authors', 'score', 'karma', 'body',
       'postedAt', 'tags', 'commentCount', 'upvoteCount', 'url',
       'contains_wanted', 'skip_authors', 'year', 'month', 'quarter',
       'alignment_forum', 'ids', 'links', 'references', 'pingback_posts'],
      dtype='object')

In [99]:
folder_path = "data/posts/"
posts = pd.DataFrame()
for file in os.listdir(folder_path):
    with open(folder_path + file) as f:
        data = json.load(f)
        posts = pd.concat([posts, pd.DataFrame(data)])
posts = posts.drop_duplicates(subset=['_id'])

In [20]:
url = "https://www.lesswrong.com/graphql"
default_headers = {
  'accept': '*/*',
  'accept-language': 'en-US,en;q=0.9',
  'content-type': 'application/json',
  'user-agent': 'AE Studio-Neglected Means Researcher tristan.tran@ae.studio',
}

# Loading posts from LessWrong

In [40]:
def get_posts(start_date:str,
              end_date: str,
              offset: int):
    """
    Get posts from LessWrong API
    :param start_date: str, start date in format'YYYY-MM-DD'
    :param end_date: str, end date in format 'YYYY-MM-DD'
    :param offset: int, offset for pagination
    """
    post_query = """
{
      posts(input: {
        terms: {
          limit:100
          view:"timeframe"
          sortedBy:"magic"
          after:"%sT07:00:00.000Z"
          before:"%sT06:59:59.999Z"
          offset: %s
          meta: null  # this seems to get both meta and non-meta posts
        }
      }
      ) {
        results {
          _id
          title
          slug
          pageUrl
          postedAt
          baseScore
          voteCount
          score
          commentCount
          meta
          question
          url
          author
          pingbacks
          coauthors {
            username
            _id
            slug
          }
          user {
            username
            _id
            slug
          }
        }
      }
    }
    """%(start_date, end_date, offset)
    response = requests.post(url, headers=default_headers, json={'query': post_query})
    # TODO implement a backoff strategy for ratelimits
    response.raise_for_status()
    return response.json()

def get_all_posts(start_date, end_date):
  """ Gets all posts from lightcone within_summary_

  Args:
      start_date (_type_): _description_
      end_date (_type_): _description_

  Returns:
      _type_: _description_
  """
  offset = 0
  results = []
  while True:
      response = get_posts(start_date, end_date, offset)
      if not response['data']['posts']['results']:
          break
      results.extend(response['data']['posts']['results'])
      offset += 100
  return results

In [41]:
r = get_posts("2021-01-01", "2021-01-02", 0)
len(r["data"]["posts"]["results"])

9

In [42]:
get_posts("2021-01-01", "2021-01-02", 8)

{'data': {'posts': {'results': [{'_id': '6kAtxZyiTdY34JWak',
     'title': 'Is Free Will A Myth?',
     'slug': 'is-free-will-a-myth',
     'pageUrl': 'https://www.lesswrong.com/posts/6kAtxZyiTdY34JWak/is-free-will-a-myth',
     'postedAt': '2021-01-01T20:56:50.221Z',
     'baseScore': 0,
     'voteCount': 0,
     'score': 0,
     'commentCount': 4,
     'meta': False,
     'question': False,
     'url': None,
     'author': None,
     'pingbacks': {},
     'coauthors': [],
     'user': {'username': 'Precious Oluwatobi Emmanuel',
      '_id': 'otR3PZ9KMsM2JyCC5',
      'slug': 'precious-oluwatobi-emmanuel'}}]}},
 'extensions': {'cacheControl': {'version': 1,
   'hints': [{'path': ['posts'], 'maxAge': 0},
    {'path': ['posts', 'results'], 'maxAge': 0},
    {'path': ['posts', 'results', 0, 'coauthors'], 'maxAge': 0},
    {'path': ['posts', 'results', 0, 'user'], 'maxAge': 0}]}}}

In [43]:
# loop that gets posts for a date range until the offset is empty
start_date = "2023-01-01"
end_date = "2023-01-02"
get_all_posts(start_date, end_date)
# this should get all posts and pingbacks

[{'_id': 'rTJrqtDLxAPxiW3sk',
  'title': 'My first year in AI alignment',
  'slug': 'my-first-year-in-ai-alignment',
  'pageUrl': 'https://www.lesswrong.com/posts/rTJrqtDLxAPxiW3sk/my-first-year-in-ai-alignment',
  'postedAt': '2023-01-02T01:28:03.470Z',
  'baseScore': 61,
  'voteCount': 33,
  'score': 0.0010619011009112,
  'commentCount': 10,
  'meta': False,
  'question': False,
  'url': None,
  'author': None,
  'pingbacks': {'Posts': ['PRMJCbBhsGgu5A6Ty']},
  'coauthors': [],
  'user': {'username': 'Alex_Altair',
   '_id': '5wu9jG4pm9q6xjZ9R',
   'slug': 'alex_altair'}},
 {'_id': 'dTWevKRiMM4ptcjjg',
  'title': 'Would it be good or bad for the US military to get involved in AI risk?',
  'slug': 'would-it-be-good-or-bad-for-the-us-military-to-get-involved',
  'pageUrl': 'https://www.lesswrong.com/posts/dTWevKRiMM4ptcjjg/would-it-be-good-or-bad-for-the-us-military-to-get-involved',
  'postedAt': '2023-01-01T19:02:30.892Z',
  'baseScore': 50,
  'voteCount': 23,
  'score': 0.0008938161

# Add Pingback if missing

In [70]:
# we can use this to add pingbacks to the posts. if they don't have them
from requests.exceptions import HTTPError
def get_all_pingback(post_id):
    """
    Worst case scenario, lightcone sends us files. They sometimes miss pingbacks or other columns
    This function gets the pingbacks for a post. If it fails, it returns an empty list.
    
    """
    pingback_query ="""{
    posts(input: {terms: {view: "pingbackPosts", postId: "%s", limit: 15, offset: 0}}) {
        results {
        _id
        extendedScore
        baseScore
        score
        voteCount
        userId
        reviewVotesHighKarma
        commentCount
        htmlBody
        }
    }
    }"""%post_id
    response = requests.post(url, headers=default_headers, json={'query': pingback_query})
    response.raise_for_status()
    return response.json()["data"]["posts"]["results"]

def get_pingback_ids(post_id):
    try:
        pingbacks = get_all_pingback(post_id)
        return [pingback["_id"] for pingback in pingbacks]
    except HTTPError as e:
        print(post_id, "returns http error")
        print(e)
        return []
    except:
        print(post_id, "returns unknown error")
        return []
def get_refs_from_pingback(df):
    ids, pingbacks = zip(*df[["_id","pingback"]].values)
    refs = {id: [] for id in ids}
    for id, pingback in zip(ids, pingbacks):
        for pb in pingback:
            if pb in refs:
                refs[pb].append(id)

    df["refs"] = df["_id"].apply(lambda x: refs[x])
    return df

In [89]:
posts.to_parquet("data/posts.parquet")

# Load comments

In [76]:
comments = pd.read_parquet("app_files/lw_comments.parquet")

In [77]:
comments.columns

Index(['htmlBody', '_id', 'postId', 'parentCommentId', 'answer', 'postedAt',
       'author_id', 'username'],
      dtype='object')

In [14]:
folder_path = "data/comments/"
comments = pd.DataFrame()
for file in os.listdir(folder_path):
    with open(folder_path + file) as f:
        data = json.load(f)
        comments = pd.concat([comments, pd.DataFrame(data)])
comments = comments.drop_duplicates(subset=['_id'])
comments.columns

Index(['_id', 'user', 'htmlBody', 'postId', 'parentCommentId', 'baseScore',
       'answer', 'postedAt'],
      dtype='object')

In [58]:
import requests
import json

url = "https://www.lesswrong.com/graphql"
comments_query = """
query multiCommentQuery($input: MultiCommentInput) {
  comments(input: $input) {
    results {
      _id
      htmlBody
      postId
      answer
      postedAt
      parentCommentId
      user {
        _id
        slug
        username
        displayName
        karma
      }
    }
  }
}
"""

def get_comments(post_id):
    payload = json.dumps({
        "operationName": "multiCommentQuery",
        "variables": {
            "input": {
                "terms": {
                    "view": "postCommentsTop",
                    "limit": 1000,
                    "postId": post_id
                },
                "enableCache": False,
                "enableTotal": True
            }
        },
        "query": comments_query
    })
    headers = {
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9',
        'content-type': 'application/json',
        'origin': 'https://www.lesswrong.com',
        'user-agent': 'AE studio negelcted approaches Research'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    return response

response = get_comments("xgrvmaLFvkFr4hKjz")

{'data': {'comments': {'results': [{'_id': 'PMtnsnkd38QNwherK', 'htmlBody': '<p>Useful post. I can expand on one point and make a minor correction. Single Particle Cryo-EM is indeed a new(ish) powerful method of protein structure elucidation starting to make an impact in drug design. It is especially useful when a protein cannot easily be crystallised to allow more straightforward X-Ray structure determination. This is usually the case with transmembrane proteins for example. However it is actually best if the protein molecules are completely unaligned in any preferred direction as the simplest application of the refinement software assumes a perfectly random 3D orientation of the many thousands of protein copies imaged on the grid. In practice this is not so easy to achieve and corrections for unwanted preferred orientation need to be made.</p>', 'postId': 'xgrvmaLFvkFr4hKjz', 'answer': False, 'postedAt': '2024-05-05T17:42:39.534Z', 'parentCommentId': None, 'user': {'_id': 'savsErdiFT

In [59]:
# call the get_comments function for each post_id and create a dataframe
import time
import tqdm
comments = pd.DataFrame()
for post_id in tqdm.tqdm(posts["_id"].values):
    response = get_comments(post_id)
    comments = pd.concat([comments, pd.DataFrame(response.json()["data"]["comments"]["results"])])
    time.sleep(1)
comments = comments.drop_duplicates(subset=['_id'])
comments.to_parquet("data/comments.parquet")


100%|██████████| 40220/40220 [16:29:12<00:00,  1.48s/it]   


In [ ]:
Index(['htmlBody', '_id', 'postId', 'parentCommentId', 'answer', 'postedAt',
       'author_id', 'username'],
      dtype='object')

In [75]:
comments = comments[["htmlBody", "_id", "postId", "parentCommentId", "answer", "postedAt", "user"]]
user_df =  pd.json_normalize(comments['user'])
user_df.rename(columns={"_id": "author_id"}, inplace=True)
comments = comments.join(user_df).drop(columns=["user"])
comments.to_parquet("app_files/lw_comments.parquet")

# create user dataframe

In [80]:
users_df = users_df.drop_duplicates(subset=["user_id"])
users_df.to_parquet("app_files/users.parquet")

# Running Similarity Score Embeddings

In [6]:
posts = pd.read_parquet("data/posts.parquet")
posts.columns

Index(['_id', 'title', 'authors', 'score', 'karma', 'body', 'postedAt', 'tags',
       'commentCount', 'upvoteCount', 'url', 'pingback', 'refs'],
      dtype='object')

In [11]:
posts[posts["upvoteCount"].istype(int)]

AttributeError: 'Series' object has no attribute 'istype'

In [3]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('AnnaWegmann/Style-Embedding')
posts.fillna("", inplace=True)
# posts["_id"] = posts.index
posts["full_text"] = posts["title"] + "\n" + posts["body"]
texts = posts["full_text"].to_list()
style_embeddings = model.encode(texts)
import torch
torch.save(style_embeddings, "app_files/style_embeddings.pt")
# each article has a style embedding
# posts["style_embedding"] = style_embeddings.tolist()

/opt/anaconda3/envs/neglected-meta-10/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


NameError: name 'torch' is not defined

In [ ]:
def get_average_author(df, author):
    average_embedding = df[df["author"]==author]["style_embedding"].mean()
    return average_embedding

def get_cosin_similarity(df, author1, author2):
    author_embedding_1 = get_average_author(df, author1)
    author_embedding_2 = get_average_author(df, author2)
    cos_sim = util.pytorch_cos_sim(author_embedding_1, author_embedding_2)
    return cos_sim

# set based approach
def cosine_similarity(set_a, set_b, df)->float:
    ...

In [35]:
posts["authors"]

0               [XFrequentist]
1       [LearnFromObservation]
2                  [Mestroyer]
3             [Gunnar_Zarncke]
4                 [DataPacRat]
                 ...          
4994                     [Zvi]
4995                      [jt]
4996       [Allison Duettmann]
4997           [Arturo Macias]
4998               [DragonGod]
Name: authors, Length: 40220, dtype: object